# Pre-Processing

In [3]:
%pip install pdfplumber
%pip install langchain
%pip install langchain_elasticsearch
%pip install langchain_huggingface
%pip install uuid
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os

#Get the file path of the pdf in the current directory
def get_pdf_path():
    list_of_files = os.listdir("./data_source/train_source")
    return list_of_files

In [34]:
import re
import pdfplumber

def pdf2text(pdf_path):
    pdf = pdfplumber.open(pdf_path)
    pages = pdf.pages
    text = ""
    for page in pages:
        sub = page.extract_text()
        text += sub

    # 특수문자 제거
    cleaned_text = re.sub(r'[^\w\s.,]', '', text)

    # .를 기준으로 문장단위 chunk 생성
    tokenized_sentence = cleaned_text.split(".")

    return tokenized_sentence

In [35]:
from uuid import uuid4

pdf_path = get_pdf_path()

parsed_text = []

for path in pdf_path:
    parsed_text.append(pdf2text("./data_source/train_source/"+path))
    

In [36]:
parsed_text

[['www',
  'kpfis',
  'or',
  'kr\n재정통계 해설\nA Guide to Fiscal Statistics재정통계 해설재정통계 해설\n총 괄 박용주 재정정보분석본부장\n기획조정 윤용중 선임연구위원\n작 성 정성호 연구위원\n심혜인 부연구위원\n김명규 부연구위원\n김선옥 부연구위원\n장윤정 부연구위원\n김상기 연구원\n조현희 연구원\n문 의 재정정보분석본부 Tel',
  ' 0269088593\n이 책은 한국재정정보원 홈페이지www',
  'kpfis',
  'or',
  'kr를 통해 보실 수 있습니다',
  '발간사\n우리나라는 경제사회적으로 많은 도전 과제에 직면해 있습니다',
  ' 저출산으로 인한 생산\n가능인구의 감소와 경제성장률 둔화, 빠른 고령화로 인한 사회적 부담 증가, 4차 산업혁\n명 진전에 따른 산업 및 고용 구조의 변화 등 다양한 영역에서 빠르고 큰 변화를 맞이하\n고 있습니다',
  ' 이런 대내외 환경 변화 속에서 지속적인 경제 성장을 이루고 개인의 삶이\n나아지기 위해서는 국가재정의 역할이 매우 중요합니다',
  '\n하지만 정부의 살림살이를 정확히 이해하는 것은 매우 어려운 일입니다',
  ' 2019년 한 해\n동안 운용되는 재정 규모는 469',
  '6조원에 이르고, 그 구조도 일반회계 뿐만 아니라 19개\n특별회계와 67개 기금으로 복잡하게 이루어져 있습니다',
  ' 또한, 국가 재정을 설명하는 용\n어도 일상 생활이나 기업 활동에서는 사용하지 않는 생소한 용어가 대부분입니다',
  '\ndBrain 예산회계시스템을 운영하는 한국재정정보원은 이 시스템에서 생산되는 재정통계\n를 비롯한 각종 재정정보를 국민들께 제공하고 있습니다',
  ' 이번에 처음 발간하게 된 재정\n통계 해설은 국가재정에 관심이 있는 일반 국민이나 학생, 국가재정 업무를 담당하는 공\n무원 등이 재정통계를 쉽게 이해하고 활용할 수 있도록 기획한 재정통계 해설서입니다',
  '\n우선, 우리나라 재정제도에 대해 제1부에서 간략히 설

In [39]:
from langchain_core.documents import Document

documents = []

for single_pdf in parsed_text:
    for sen in single_pdf:
        doc = Document(
            page_content=sen,
            metadata={"source": f"{pdf_path}"},
        )
        documents.append(doc)

uuids = [str(uuid4()) for _ in range(len(documents))]

In [40]:
documents

[Document(metadata={'source': "['재정통계해설.pdf', '1-1 2024 주요 재정통계 1권.pdf', '국토교통부_민간임대(융자).pdf', '고용노동부_내일배움카드(일반).pdf', '2024년도 성과계획서(총괄편).pdf', '2024 나라살림 예산개요.pdf', '보건복지부_노인일자리 및 사회활동지원.pdf', '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf', '국토교통부_전세임대(융자).pdf', '「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf', '국토교통부_소규모주택정비사업.pdf', '고용노동부_청년일자리창출지원.pdf', '월간 나라재정 2023년 12월호.pdf', '보건복지부_생계급여.pdf', '중소벤처기업부_창업사업화지원.pdf', '고용노동부_조기재취업수당.pdf']"}, page_content='www'),
 Document(metadata={'source': "['재정통계해설.pdf', '1-1 2024 주요 재정통계 1권.pdf', '국토교통부_민간임대(융자).pdf', '고용노동부_내일배움카드(일반).pdf', '2024년도 성과계획서(총괄편).pdf', '2024 나라살림 예산개요.pdf', '보건복지부_노인일자리 및 사회활동

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchStore

In [27]:
embeddings = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-mpnet-base-v2",
    # model_kwargs={'device':'cuda'}
)

client = ElasticsearchStore(
    es_url="https://agi.veritasgarage.com:443", 
    index_name="dacon_nori_index", 
    es_user="elastic", 
    es_password="sukbeom", 
    embedding=embeddings)

client.add_documents(documents=documents, ids=uuids)

['16ba3406-97af-4db2-8bbb-a94354acaa63',
 'd9d56d15-8dd5-47cd-8797-340325ba740c',
 '04de5bfd-9538-49dd-bd17-57348db2b2df',
 '8fa75948-8fcd-4df7-9a07-21567f09a24a',
 '012f8b88-951d-47a4-b978-e5ee85be3870',
 '01b7aa81-19e9-46a8-8b31-f89e1d5b7914',
 '3fb22421-7a23-48c6-aba5-8475a3222a0f',
 '7a697a18-8997-4bdd-8952-cdc2c972ffd0',
 '8f044b9d-3736-4c74-868f-aaaabe81acde',
 '1889266b-0157-45cb-b91b-1262175c1197',
 '29b1b210-e750-487a-b37b-7cee210a0f89',
 '9e01ab6a-1ca5-474b-a906-5e6d832776cf',
 '3745bf22-0c10-427d-89b9-8f9122a7773e',
 'c0c29c38-16fa-4306-9e82-901a71a7883c',
 '1cb09d03-04a3-42bc-8101-89c9995516db',
 'ec2906be-2ae7-4014-9afe-15b0014e2bac',
 '26cfb57a-fb04-49de-ac85-6752ab04500a',
 '0d363ad6-702d-4fd0-851a-966baf0bf297',
 '6f872961-c707-47d4-ab57-bfb5013d354c',
 '551b5eff-c4f0-4c85-b18f-80a6dfd1f3a6',
 '209a076e-786f-402d-b1e5-03b0fcd3b40e',
 'fcdab96a-8367-41b3-8ea7-7036f608407d',
 '200f5a38-a175-4464-8d5e-e8dfce218616',
 'e9ed2f0d-bec7-4b9a-b8c7-71563256f6da',
 '6da446d9-5936-

In [28]:
def showResults(output):
  print("Total results: ", len(output))
  for index in range(len(output)):
    print(output[index])

In [31]:
query = "민간임대"
result = client.similarity_search(query=query)

showResults(result)

Total results:  4
page_content=' 공정투명한 국가공무원 채용
①국가공무원 채용시험
일반
인재채용 운영에 대한 응시자 종 1' metadata={'source': "['재정통계해설.pdf', '1-1 2024 주요 재정통계 1권.pdf', '국토교통부_민간임대(융자).pdf', '고용노동부_내일배움카드(일반).pdf', '2024년도 성과계획서(총괄편).pdf', '2024 나라살림 예산개요.pdf', '보건복지부_노인일자리 및 사회활동지원.pdf', '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf', '국토교통부_전세임대(융자).pdf', '「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf', '국토교통부_소규모주택정비사업.pdf', '고용노동부_청년일자리창출지원.pdf', '월간 나라재정 2023년 12월호.pdf', '보건복지부_생계급여.pdf', '중소벤처기업부_창업사업화지원.pdf', '고용노동부_조기재취업수당.pdf']"}
page_content='
①유도무기사업의 적기 일반 기관대표
유도무기사업 1' metadata={'source': "['재정통계해설.pdf', '1-1 2024 주요 재정통계 1권.pdf', '국토교통부_민간임대(융자).pdf', '고용노동부_내일배움카드(일반).pdf', '2024년도 성과계획서(총ᄀ